<a href="https://colab.research.google.com/github/anshupandey/prompt_engineering/blob/main/code8_prompt_evaluation_tracing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt Evaluation and Tracing with MLflow
This notebook is designed to help you evaluate and trace prompts using MLflow. It provides a structured way to log prompt evaluations, track their performance, and visualize the results over time.

Key Benefits of MLflow Prompt Evaluation
* Effective Evaluation: `MLflow's LLM Evaluation API provides a simple and consistent way to evaluate prompts across different models and datasets without writing boilerplate code.
* Compare Results: Compare evaluation results with ease in the MLflow UI.
* Tracking Results: Track evaluation results in MLflow Experiment to maintain the history of prompt performance and different evaluation settings.
* Tracing: Inspect model behavior during inference deeply with traces generated during evaluation.

In [1]:
!pip install mlflow openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.2/676.2 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 13.3 MB/s eta 0:00:00


In [4]:
import os
import mlflow
from openai import OpenAI
os.environ['OPENAI_API_KEY'] = "sk-proj-xx-xxx-xx-xx-xx"
mlflow.set_tracking_uri("http://20.80.248.210:5000/")

### 1. Create a new prompt

In [5]:
import mlflow

# Use double curly braces for variables in the template
initial_template = """\
Summarize content you are provided with in {{ num_sentences }} sentences.

Sentences: {{ sentences }}
"""

# Register a new prompt
prompt = mlflow.register_prompt(
    name="anshu-summarization-prompt",
    template=initial_template,
    # Optional: Provide a commit message to describe the changes
    commit_message="Initial commit",
)

# The prompt object contains information about the registered prompt
print(f"Created prompt '{prompt.name}' (version {prompt.version})")

/tmp/ipython-input-5-2441474764.py:11: FutureWarning: The `mlflow.register_prompt` API is moved to the `mlflow.genai` namespace. Please use `mlflow.genai.register_prompt` instead. The original API will be removed in the future release.
  prompt = mlflow.register_prompt(
2025/07/18 12:02:53 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: anshu-summarization-prompt, version 3


Created prompt 'anshu-summarization-prompt' (version 3)


### 2. Prepare Evaluation Data

In [6]:
import pandas as pd

eval_data = pd.DataFrame(
    {
        "inputs": [
            "Artificial intelligence has transformed how businesses operate in the 21st century. Companies are leveraging AI for everything from customer service to supply chain optimization. The technology enables automation of routine tasks, freeing human workers for more creative endeavors. However, concerns about job displacement and ethical implications remain significant. Many experts argue that AI will ultimately create more jobs than it eliminates, though the transition may be challenging.",
            "Climate change continues to affect ecosystems worldwide at an alarming rate. Rising global temperatures have led to more frequent extreme weather events including hurricanes, floods, and wildfires. Polar ice caps are melting faster than predicted, contributing to sea level rise that threatens coastal communities. Scientists warn that without immediate and dramatic reductions in greenhouse gas emissions, many of these changes may become irreversible. International cooperation remains essential but politically challenging.",
            "The human genome project was completed in 2003 after 13 years of international collaborative research. It successfully mapped all of the genes of the human genome, approximately 20,000-25,000 genes in total. The project cost nearly $3 billion but has enabled countless medical advances and spawned new fields like pharmacogenomics. The knowledge gained has dramatically improved our understanding of genetic diseases and opened pathways to personalized medicine. Today, a complete human genome can be sequenced in under a day for about $1,000.",
            "Remote work adoption accelerated dramatically during the COVID-19 pandemic. Organizations that had previously resisted flexible work arrangements were forced to implement digital collaboration tools and virtual workflows. Many companies reported surprising productivity gains, though concerns about company culture and collaboration persisted. After the pandemic, a hybrid model emerged as the preferred approach for many businesses, combining in-office and remote work. This shift has profound implications for urban planning, commercial real estate, and work-life balance.",
            "Quantum computing represents a fundamental shift in computational capability. Unlike classical computers that use bits as either 0 or 1, quantum computers use quantum bits or qubits that can exist in multiple states simultaneously. This property, known as superposition, theoretically allows quantum computers to solve certain problems exponentially faster than classical computers. Major technology companies and governments are investing billions in quantum research. Fields like cryptography, material science, and drug discovery are expected to be revolutionized once quantum computers reach practical scale.",
        ],
        "targets": [
            "AI has revolutionized business operations through automation and optimization, though ethical concerns about job displacement persist alongside predictions that AI will ultimately create more employment opportunities than it eliminates.",
            "Climate change is causing accelerating environmental damage through extreme weather events and melting ice caps, with scientists warning that without immediate reduction in greenhouse gas emissions, many changes may become irreversible.",
            "The Human Genome Project, completed in 2003, mapped approximately 20,000-25,000 human genes at a cost of $3 billion, enabling medical advances, improving understanding of genetic diseases, and establishing the foundation for personalized medicine.",
            "The COVID-19 pandemic forced widespread adoption of remote work, revealing unexpected productivity benefits despite collaboration challenges, and resulting in a hybrid work model that impacts urban planning, real estate, and work-life balance.",
            "Quantum computing uses qubits existing in multiple simultaneous states to potentially solve certain problems exponentially faster than classical computers, with major investment from tech companies and governments anticipating revolutionary applications in cryptography, materials science, and pharmaceutical research.",
        ],
    }
)

### 3. Define prediction function

In [7]:
client = OpenAI()

def predict(data: pd.DataFrame) -> list[str]:
    predictions = []
    prompt = mlflow.load_prompt("prompts:/anshu-summarization-prompt/3")

    for _, row in data.iterrows():
        # Fill in variables in the prompt template
        content = prompt.format(sentences=row["inputs"], num_sentences=1)
        completion = client.responses.create(
            model="gpt-4o-mini",
            input=[{"role": "user", "content": content}],
            temperature=0.1,
        )
        predictions.append(completion.output_text)

    return predictions

### 4. Run Evaluation

In [10]:

with mlflow.start_run(run_name="anshu-prompt-evaluation"):
    mlflow.log_param("model", "gpt-4o-mini")
    mlflow.log_param("temperature", 0.1)

    results = mlflow.evaluate(
        model=predict,
        data=eval_data,
        targets="targets",
        extra_metrics=[
            mlflow.metrics.latency(),
            # Specify GPT4 as a judge model for answer similarity. Other models such as Anthropic,
            # Bedrock, Databricks, are also supported.
            mlflow.metrics.genai.answer_similarity(model="openai:/gpt-4o"),
            mlflow.metrics.toxicity(),
            mlflow.metrics.flesch_kincaid_grade_level()
        ],
    )

2025/07/18 12:27:32 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
/tmp/ipython-input-7-3931437831.py:5: FutureWarning: The `mlflow.load_prompt` API is moved to the `mlflow.genai` namespace. Please use `mlflow.genai.load_prompt` instead. The original API will be removed in the future release.
  prompt = mlflow.load_prompt("prompts:/anshu-summarization-prompt/3")
/tmp/ipython-input-7-3931437831.py:5: FutureWarning: The `mlflow.load_prompt` API is moved to the `mlflow.genai` namespace. Please use `mlflow.genai.load_prompt` instead. The original API will be removed in the future release.
  prompt = mlflow.load_prompt("prompts:/anshu-summarization-prompt/3")
/tmp/ipython-input-7-3931437831.py:5: FutureWarning: The `mlflow.load_prompt` API is moved to the `mlflow.genai` namespace. Please use `mlflow.genai.load_prompt` instead. The original API will be removed in the future release.
  prompt = mlflow.load_prompt("prompts:/anshu-summarization-prompt/3")
/tmp/ip

  0%|          | 0/1 [00:00<?, ?it/s]

2025/07/18 12:27:42 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/07/18 12:27:42 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
2025/07/18 12:27:42 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/07/18 12:27:42 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'flesch_kincaid_grade_level' at index 2 in the `extra_metrics` parameter because it returned None.


  0%|          | 0/5 [00:00<?, ?it/s]

2025/07/18 12:27:45 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/07/18 12:27:45 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
2025/07/18 12:27:45 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/07/18 12:27:45 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'flesch_kincaid_grade_level' at index 2 in the `extra_metrics` parameter because it returned None.


🏃 View run anshu-prompt-evaluation at: http://20.80.248.210:5000/#/experiments/0/runs/dbc4f300bb5a480b81e06b55c5d6c7ef
🧪 View experiment at: http://20.80.248.210:5000/#/experiments/0


[Trace(trace_id=25a44e53690642ea8792e92c6a4654cc), Trace(trace_id=30cae5ea401e4be2af6f2d604ceaee08), Trace(trace_id=62dc5c2d9a5541059269e52da5633cde), Trace(trace_id=41aa9279e6694cbeb19a9e8eaf1a65d4), Trace(trace_id=cfef5db190af486ea006116f90869c2e)]

### 5. View Results
<img src="https://mlflow.org/docs/latest/assets/images/prompt-evaluation-result-7c106f17187fdc750439725d086c389b.png" alt="MLflow LLM Evaluation UI" width="800">

<img src = "https://mlflow.org/docs/latest/assets/images/prompt-evaluation-chart-8a93612e37184b8279c699fd6640013d.png" >

## Tracing

In [11]:
mlflow.openai.autolog()

with mlflow.start_run(run_name="anshu123"):
  response = client.responses.create(model='gpt-4o',
                                     input='What is quantum computing?')
  print(response.output_text)

Quantum computing is an advanced field of computing that leverages the principles of quantum mechanics to process information in fundamentally different ways than classical computers. Here are some key aspects:

1. **Qubits**: Unlike classical bits, which can be either 0 or 1, qubits can exist in superpositions, where they hold both 0 and 1 simultaneously. This property allows quantum computers to process a vast amount of possibilities at once.

2. **Entanglement**: This is a quantum phenomenon where qubits become interconnected such that the state of one qubit can depend on the state of another, no matter how far apart they are. Entanglement enables complex computations that are difficult for classical computers.

3. **Quantum Gates**: Quantum computations are conducted using quantum gates, which manipulate qubits through quantum operations. These gates differ significantly from classical logic gates due to their ability to maintain superpositions and entanglements.

4. **Quantum Deco

Trace(trace_id=75fcdf0c7270431e9c04e8135ec3a5fc)

## Prompt Optimization

In [16]:
import os
from typing import Any
import mlflow
from mlflow.genai.scorers import scorer
from mlflow.genai.optimize import OptimizerConfig, LLMParams

In [13]:
# Register the initial prompt
initial_template = """
Answer to this math question: {{question}}.
Return the result in a JSON string in the format of {"answer": "xxx"}.
"""

prompt = mlflow.genai.register_prompt(
    name="anshu-math",
    template=initial_template,
)


2025/07/18 12:35:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: anshu-math, version 1


In [14]:

# The data can be a list of dictionaries, a pandas DataFrame, or an mlflow.genai.EvaluationDataset
# It needs to contain inputs and expectations where each row is a dictionary.
train_data = [
    {
        "inputs": {"question": "Given that $y=3$, evaluate $(1+y)^y$."},
        "expectations": {"answer": "64"},
    },
    {
        "inputs": {
            "question": "The midpoint of the line segment between $(x,y)$ and $(-9,1)$ is $(3,-5)$. Find $(x,y)$."
        },
        "expectations": {"answer": "(15,-11)"},
    },
    {
        "inputs": {
            "question": "What is the value of $b$ if $5^b + 5^b + 5^b + 5^b + 5^b = 625^{(b-1)}$? Express your answer as a common fraction."
        },
        "expectations": {"answer": "\\frac{5}{3}"},
    },
    {
        "inputs": {"question": "Evaluate the expression $a^3\\cdot a^2$ if $a= 5$."},
        "expectations": {"answer": "3125"},
    },
    {
        "inputs": {"question": "Evaluate $\\lceil 8.8 \\rceil+\\lceil -8.8 \\rceil$."},
        "expectations": {"answer": "17"},
    },
]

eval_data = [
    {
        "inputs": {
            "question": "The sum of 27 consecutive positive integers is $3^7$. What is their median?"
        },
        "expectations": {"answer": "81"},
    },
    {
        "inputs": {"question": "What is the value of $x$ if $x^2 - 10x + 25 = 0$?"},
        "expectations": {"answer": "5"},
    },
    {
        "inputs": {
            "question": "If $a\\ast b = 2a+5b-ab$, what is the value of $3\\ast10$?"
        },
        "expectations": {"answer": "26"},
    },
    {
        "inputs": {
            "question": "Given that $-4$ is a solution to $x^2 + bx -36 = 0$, what is the value of $b$?"
        },
        "expectations": {"answer": "-5"},
    },
]


In [17]:

# Define a custom scorer function to evaluate prompt performance with the @scorer decorator.
# The scorer function for optimization can take inputs, outputs, and expectations.
@scorer
def exact_match(expectations: dict[str, Any], outputs: dict[str, Any]) -> bool:
    return expectations["answer"] == outputs["answer"]



In [19]:
!pip install dspy --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2

In [20]:
# Optimize the prompt
result = mlflow.genai.optimize_prompt(
    target_llm_params=LLMParams(model_name="openai/gpt-4.1-mini"),
    prompt=prompt,
    train_data=train_data,
    eval_data=eval_data,
    scorers=[exact_match],
    optimizer_config=OptimizerConfig(
        num_instruction_candidates=8,
        max_few_show_examples=2,
    ),
)

# The optimized prompt is automatically registered as a new version
print(result.prompt.uri)

2025/07/18 12:37:02 INFO mlflow.genai.optimize.optimizers.dspy_mipro_optimizer: Started optimizing prompt prompts:/anshu-math/1. Please wait as this process typically takes several minutes, but can take longer with large datasets...
2025/07/18 12:38:34 INFO mlflow.genai.optimize.optimizers.dspy_mipro_optimizer: Prompt optimization completed. Evaluation score changed from 25.0 to 50.0.
/usr/local/lib/python3.11/dist-packages/mlflow/genai/optimize/optimizers/dspy_mipro_optimizer.py:116: FutureWarning: The `mlflow.register_prompt` API is moved to the `mlflow.genai` namespace. Please use `mlflow.genai.register_prompt` instead. The original API will be removed in the future release.
  optimized_prompt = register_prompt(
2025/07/18 12:38:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: anshu-math, version 2


prompts:/anshu-math/2


[Trace(trace_id=fbdfecf5480d4e7b8fda34aac435a768), Trace(trace_id=c5aafa1571904e579d69eaa5f56f3a27), Trace(trace_id=93b0cafd3d3c40c9a7f2f9b4d5892306), Trace(trace_id=c274b1f18e03473aa5b99cce516ade15), Trace(trace_id=4ba8528b7b494495bb14d546d2236fe6), Trace(trace_id=1ed09f6938dc479a916bcff35d8f9c10), Trace(trace_id=05418114f595419f9f7659f72aa821c0), Trace(trace_id=e63a73cdd3dd4461b9c393fb7cce45f6), Trace(trace_id=ce6be1386de140d58ec3c72c076367db), Trace(trace_id=3b2211403737488bb7939a8fec40555d)]